---
#### Data Analysis
- 正则  
- bs4  
- xpath
---
#### 常用正则表达式
> 单字符：
- . ：处换行外所有字符  
- [] ：[aoe] [a-w] 匹配集合中任意一个字符  
- \d ：数字 [0-9]  
- \D ：非数字  
- \w ：数字、字母、下划线、中文  
- \W ：非\w  
- \s ：所有非空白字符，等价于[\f\n\r\t\v]  
- \S ：非空白  
> 数量修饰：
- \* ：任意多次，>=0  
- \+ ：至少1次 >=1  
- ? ：可有可无，0次或者1次  
- {m} ：固定m次 hello{3,}  
- {m,} ：至少m次  
- {m,n} ：至少m-n次  
> 边界：
- $ ：以某某结尾
- ^ ：以某某开头
> 分组：
- (ab)
- 贪婪模式：.*
- 非贪婪模式(惰性)模式：.*?
- re.I ：忽略大小写
- re.M ：多行匹配
- re.S ：单行匹配
- re.sub(正则表达式，替换内容，字符串)
---
#### Mission 1：常用正则练习

In [18]:
import re
# 提取出Python
key='javapythonc++php'
res=re.findall('python',key)[0]
print(res)
###################################################################
# 提取出hello word
key='<html><h1>hello world<h1></html>'
res=re.findall('<h1>(.*)<h1>',key)[0]
print(res)
###################################################################
# 提取出170
string='我身高是170'
res=re.findall('\d+',string)
print(res)
###################################################################
# 提取出http://和https://
key='http://www.baidu.com and https://boob.com'
res=re.findall('https?://',key)
print(res)
###################################################################
# 提取出hello
key='lalala<html>hello</html>hahah'
res=re.findall('<[Hh][Tt][mM][lL]>(.*)</[Hh][Tt][mM][lL]>',key)[0]
print(res)
###################################################################
# 提取出hit.
key='bobo@hit.edu.com'
res=re.findall('h.*?\.',key)[0]
print(res)
###################################################################
# 提取出sas和saas
key='saas and sas and saaaas'
res=re.findall('sa{1,2}s',key)
print(res)

python
hello world
['170']
['http://', 'https://']
hello
hit.
['saas', 'sas']


---
#### Mission 2：爬取图片数据
- text——字符串型数据  
- content——2进制型数据  
- json——对象

In [2]:
import requests
if __name__ == "__main__":
    headers={
        'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36'
    }
    url = 'https://tse1-mm.cn.bing.net/th/id/R-C.47d5fd52caf4fce40d910c06a7282572?rik=OKBN%2btbwWi3Gxw&riu=http%3a%2f%2fwww.sa.buaa.edu.cn%2f__local%2f4%2f7D%2f5F%2fD52CAF4FCE40D910C06A7282572_411891C6_7668.jpg&ehk=O84%2fcaojbXEwILEhWQwc9ZCtqddou1P4SxY3upbS4g0%3d&risl=&pid=ImgRaw&r=0'
    img_data = requests.get(url=url,headers=headers).content
    with open('hallThruster.jpg','wb') as fp:
        fp.write(img_data)

---
#### Mission 3：批量获取图片数据——正则方法
- step 1：通过爬虫获取整个页面  
- step 2：正则解析数据获取需要信息所在地址  
- step 3：根据所需信息地址发起请求获取响应数据  
- step 4：持久化存储

In [12]:
import requests
import os
import re
import time
if __name__ == "__main__":
    #UA伪装
    headers={
        'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36'
    }
    if not os.path.exists('./picture_libs'):#如果路径不存在，则用mkdir创建一个单级目录
        os.mkdir('./picture_libs')        
    #这个最大的for循环是用来执行翻页操作的，由于页数不知道，我这边是随便指定结束页尾的
    for pageNum in range(1, 6):
        # 当你看到输出语句存在 %d， %s， %f等百分号的时候，就接触到python字符串格式化输出相关知识。
        url = 'https://desk.zol.com.cn/fengjing/%d.html'#1.指定url
        new_url = format(url%pageNum)#学习format函数的用法
        #2.发起请求    3.获取响应数据
        page_text = requests.get(url=new_url, headers=headers).text

        #存储以及打印出page_text中的内容
        # with open('./content','w',encoding='utf-8') as fp:
        #     fp.write(page_text)
        #     fp.write('\n')#换行

        #正则数据解析
        ex = 'class="photo-list-.*?src="(.*?)"\stitle\s=\s".*?".*?<ins>.*?</ins>'
        # 根据正则表达式ex提取page_text中的内容，re.S为单行匹配
        img_src_list = re.findall(ex, page_text, re.S)
        # print(img_src_list)
        i = 0
        for src in img_src_list:#遍历img_src_list，即1.指定url
            #2.发起请求 3.获取响应数据，注意这里的是图片数据，用的是content
            img_data = requests.get(url=src, headers=headers).content#对每一个
            # print(img_data)
            # 将图片地址以字符‘/’分割，最后一个字符串为图片原来地址，为方便区分人为指定图片名称
            # img_name=src.split('/')[-1] 
            i = i + 1
            str2 = str(pageNum)+'_'+str(i)#图片的定位是页数+在这页的位置
            img_name = str2+'.jpg'#图片名称
            img_path = './picture_libs/'+img_name#图片路径
            with open(img_path, 'wb', )as fp:#4.持久化存储
                fp.write(img_data)
                time.sleep(0.5)
            # print(img_name,'下载成功！')
    print('图片下载完成！')


图片下载完成！


---
#### Mission 4：bs4数据解析(仅限Python)，环境：```pip install bs4```，```pip install lxml```
> 数据解析原理：
>> 1、标签定位；<br>
>> 2、提取标签、标签属性中存储的数据值；

> bs4数据解析原理：
>> 1、实例化一个BeautifulSoup对象，并且将页面源码数据加载至该对象中；<br>
>> 2、通过调用BeautifulSoup对象中相关属性或方法进行标签定位和数据提取；

> 如何实例化一个BeautifulSoup对象：
>> 1、将本地html文档中的数据加载到该对象中，数据加载与实例化同步实现：
>> ```fp=open('test.html','r',encoding='utf-8')```<br>
>> ```soup=BeautifulSoup(fp,'lxml')```<br>
>> 2、将互联网上提取的页面源码加载至该对象中(使用较多)：<br>
>> ```page_text=response.text```<br>
>> ```soup=BeautifulSoup(page_text,'lxml')```<br>

> bs4中提供的数据解析的方法与属性
>> 1、soup.tagName：返回文档中第1次出现的tagNam标签对应的标签内容，如：```soup.div```；<bR>
>> 2、```soup.find()```：
>>> - ```soup.find('tagName')```：等价于```soup.tagName```。
>>> - ```soup.find('tagName',class_/id/attr='attri')```：根据属性定位tagName标签中具体内容；<br>

>> 3、```soup.find_all('tagName')```：找到所有的符合标签的内容，返回为列表

>> 4、select：<br>
>>> ```soup.select('某种选择器(id,class,标签···选择器)')```，返回列表；
>>> 层级选择器：<br>  
>>> - ```soup.select('.tang > ul > li > a')```: >表示的是一个层级；
>>> - ```soup.select('.tang > ul a')```: 空格表示的是多个层级；

>> 5、获取标签中的文本数据
>>> ```soup.tagName.text/string/get_text()```
>>> - ```text/get_text()```：获得标签中所有的文本内容
>>> - ```string```：仅可以获得获得标签下直系的文本内容，若没有内容返回None

>> 6、获取标签中的属性值
>>> - ```soup.tageName['属性标签']```
---
#### Misssion 5：采用bs4下载书籍章节

In [ ]:
import requests
from bs4 import BeautifulSoup
import time
if __name__ == "__main__":
    headers={
        'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36'
    }
    url = 'https://www.shicimingju.com/book/sanguoyanyi.html'
    # 为了避免使用text出现乱码，这里采用2进制获取相关内容
    page_text = requests.get(url=url,headers=headers).content
    soup = BeautifulSoup(page_text,'lxml')
    li_list = soup.select('.book-mulu > ul > li')
    # print(li_list)
    fp = open('sanguo.txt','w',encoding='utf-8')
    for li in li_list:
        title = li.a.string
        detail_url = 'https://www.shicimingju.com/' + li.a['href']
        # print(title)
        # print(detail_url,'\n')
        chapter = requests.get(url=detail_url,headers=headers).content
        detail_soup = BeautifulSoup(chapter,'lxml')
        div_tag = detail_soup.find('div',class_='chapter_content')
        content = div_tag.text
        fp.write(title+':'+content+'\n\n')
        print('<<'+title+'>>'+'下载完毕！')
        time.sleep(1)


---
#### Mission 6：xpath数据解析(常用、便捷、高效、通用性强)，环境：```pip install lxml```
##### xpath数据解析原理
- 1、实例化一个etree对象，并将需要被解析的页面源码数据加载至该对象中；  
```tree = etree.parse('test.html')```  
- 2、通过调用etree对象中xpath方法结合xpath表达式实现标签定位和内容捕获；
##### etree对象实例化
- 1、将本地的html文档中的源码数据加载到etree对象中：  
```etree.parse(filePath)```
- 2、将互联网上获取的源码数据加载到该对象中：  
```etree.HTML('page_text')```
##### xpath数据解析主要使用xpath表达式：```xpath('xpath表达式')```
- 1、/：表示从根节点开始定位，表示单个层级。```r=etree.xpath('/html/body/div')```返回Element列表
- 2、//：表示多个层级。可以表示从任意位置开始定位。```r=etree.xpath('/html//div')```或者```r=etree.xpath('//div')```
- 3、属性定位(```tagName[@attrName="attrValue"]```)：<br>
```r=tree.xpath('//div[@class="song"]')```
- 4、索引定位(索引是从1开始的)：```r=tree.xpath('//div[@class="song"]/p[3]')```
- > 5、提取文本内容
- >> - ```/text()```：获取标签中直系文本内容```r=tree.xpath('//div[@class="tang"]//li[5]/a/text()')[0]```
- >> - ```//text()```：获取标签下所有的文本内容```r=tree.xpath('//div[@class="tang"]//li[5]/a//text()')```
- 6、提取属性(tagName/@arrtName)：```r = tree.xpath('//div[@class="song"]/img/@src')```

---
#### Misssion 7：xpath实战：爬取58二手房中的房源信息

In [36]:
import requests
from lxml import etree
if __name__ == "__main__":
    headers={
        'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36'
    }
    url = 'https://bj.58.com/ershoufang/'
    page_text = requests.get(url=url,headers=headers).text
    tree = etree.HTML(page_text)
    div_list = tree.xpath('//div[@id="__layout"]//section[@class="list"][1]')
    # print(div_list)
    fp = open('58.txt','w',encoding='utf-8')
    for div in div_list:
        title = div.xpath('.//div[@class="property-content-title"]/h3/text()') # 局部解析相对路径
        for content in title:
            fp.write(content+'\n')
    print('Done！')

Done！


---
#### Mission 8：解析下载4K图片数据